In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import config as cfg
import seaborn as sns
from data import hf_data, t_data

# Heart Failure Dataset

## Sparsity

In [ ]:
columns = hf_data.select_dtypes(include='number').columns
rows, cols = len(columns)-1, len(columns)-1
plt.figure()
fig, axs = plt.subplots(rows, cols, figsize=(cols*4, rows*4), squeeze=False)
for i in range(len(columns)):
    var1 = columns[i]
    for j in range(i+1, len(columns)):
        var2 = columns[j]
        axs[i, j-1].set_title("%s x %s"%(var1,var2))
        axs[i, j-1].set_xlabel(var1)
        axs[i, j-1].set_ylabel(var2)
        axs[i, j-1].scatter(hf_data[var1], hf_data[var2])
plt.show()

## Correlation

In [ ]:
fig = plt.figure(figsize=[12, 12])
corr_mtx = hf_data.corr()
sns.heatmap(corr_mtx, xticklabels=corr_mtx.columns, yticklabels=corr_mtx.columns, annot=True, cmap='Blues')
plt.title('Heart Failure Correlation analysis')
plt.show()

# Toxicity Dataset

## Sparsity

All of the variables are binary and so there is no sparsity analysis to be done, since the scatterplot will always just show a square.

## Correlation

In [ ]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

cp_data = t_data.copy()
most_cor_25 = get_top_abs_correlations(cp_data.drop(columns=['toxic']), 25)
print(most_cor_25)
most_cor = most_cor_25[:7]
print("done")

In [ ]:
columns_tox = []
for i in range(len(most_cor.index)):
    columns_tox += [most_cor.index[i][0], most_cor.index[i][1]]
columns_tox = sorted(list(dict.fromkeys(columns_tox)))

corr_mtx_toxicity = t_data.corr().loc[columns_tox, columns_tox]
print(corr_mtx_toxicity)



In [ ]:
fig = plt.figure(figsize=(8, 8))
sns.heatmap(corr_mtx_toxicity, xticklabels=columns_tox, yticklabels=columns_tox, annot=True, cmap='Blues')
plt.yticks(rotation=0)
plt.title('Toxicity Correlation analysis')
plt.show()